In [1]:
import os
import numpy as np
import pandas as pd
import uuid
import time
from collections import defaultdict
from random import shuffle
from typing import List

from librep.actions.action import Action
from librep.actions.spectogram import ExtractSpectrogram, ExtractCombinedSpectrogram, PlotSpectrogram, ExtractCombinedLogSpectrogram

In [2]:
class RemoveNanAlongDimensions(Action):
    def run(self, input_data, metadata: dict):
        arr = []
        for i in range(len(input_data)):
            arr.append(input_data[i, ~np.isnan(input_data[i])])
        return (np.stack(arr), metadata)
    
class SplitToLists(Action):
    def __init__(self, start: int = None, end: int = None):
        self.start = start
        self.end = end
    
    def run(self, input_data, metadata: dict):
        start = self.start if self.start is not None else 0
        end = self.end if self.end is not None else len(input_data)
        if start >= end:
            raise ValueError("End value must be greater than start value")
        
        return ([input_data[i] for i in range(start, end)], metadata)
    
class PlotAndSave(Action):
    def __init__(self, plotter: PlotSpectrogram, root_output_path: str):
        self.plotter = plotter
        self.output_path = root_output_path
        
    def run(self, input_data, metadata: dict):
        path = os.path.join(
            self.output_path, 
            f"{metadata['class']}-{metadata['user']}-{metadata['serial']}-{metadata['index']}.png"
        )
        os.makedirs(self.output_path, exist_ok=True)
        self.plotter.output_path = path
        res = self.plotter.run(input_data, metadata)
        #print(f"Figure saved to '{path}'")
        return res
    
from tqdm import tqdm

class ActionExecutor:
    def __init__(self, description, actions, input_data, metadata):
        self.description = description
        self.actions = actions
        self.input_data = input_data
        self.metadata = metadata
        
    def run(self):
        print(f"Executing '{self.description}'")
        for i in tqdm(range(len(self.input_data))):
            (res, meta) = (self.input_data[i], self.metadata.loc[i])
            for action in self.actions:
                (res, meta) = action.run(res, meta)
        return True
    
def read_data_meta(directory, datafile="data.npz", metafile="meta.csv"):
    data_fname = os.path.join(directory, datafile)
    meta_fname = os.path.join(directory, metafile)
    data = np.load(data_fname)["data"]
    meta = pd.read_csv(meta_fname)
    return data, meta

In [3]:
uid = str(time.time())[:8]

# Raw spectrogram combined

In [8]:
for t in ["accelerometer", "gyroscope"]:
    for desc, datafile, metafile in [("train", "train.npz", "train.csv"), ("validation", "validation.npz", "validation.csv"), ("test", "test.npz", "test.csv"), ("direct", "data.npz", "meta.csv")]:
        for nperseg, nooverlap, vmin, vmax in [(256, 128, 0.0, 0.0000001)]:
            descname = f"10log10-{desc}-{nperseg}-{nooverlap}-{vmin}-{vmax}"
            dataset_dir = os.path.join("data", "embbedings", "kuhar", "direct", "raw", t, "486c8aad")
            output_path = os.path.join("data", "embbedings", "kuhar", "spectrogram", "raw", f"{t}-combined", "486c8aad", descname) 
            data, metadata = read_data_meta(dataset_dir, datafile=datafile, metafile=metafile)

            remover_nans = RemoveNanAlongDimensions()
            splitter = SplitToLists()
            spec_extractor = ExtractCombinedLogSpectrogram(fs=100, nperseg=nperseg, nooverlap=nooverlap)
            plotter = PlotSpectrogram(vmin=vmin, vmax=vmax, figsize=(8,8), plot_axis=False, display=False)
            plot_and_save = PlotAndSave(plotter, output_path) 
            actions = [remover_nans, splitter, spec_extractor, plot_and_save]
            executor = ActionExecutor(
                description=f"Extract combined spectrogram from {t} to output: '{output_path}'",
                actions=actions,
                input_data=data,
                metadata=metadata
            )
            executor.run()

  0%|          | 1/216 [00:00<00:32,  6.62it/s]

Executing 'Extract combined spectrogram from accelerometer to output: 'data/embbedings/kuhar/spectrogram/raw/accelerometer-combined/486c8aad/10log10-train-256-128-0.0-1e-07''


  3%|▎         | 1/36 [00:00<00:05,  6.56it/s]

Executing 'Extract combined spectrogram from accelerometer to output: 'data/embbedings/kuhar/spectrogram/raw/accelerometer-combined/486c8aad/10log10-validation-256-128-0.0-1e-07''


  1%|          | 1/90 [00:00<00:13,  6.60it/s]

...

  1%|          | 1/90 [00:00<00:13,  6.66it/s]

Executing 'Extract combined spectrogram from gyroscope to output: 'data/embbedings/kuhar/spectrogram/raw/gyroscope-combined/486c8aad/10log10-test-256-128-0.0-1e-07''


  0%|          | 1/1945 [00:00<04:08,  7.81it/s]

Executing 'Extract combined spectrogram from gyroscope to output: 'data/embbedings/kuhar/spectrogram/raw/gyroscope-combined/486c8aad/10log10-direct-256-128-0.0-1e-07''


100%|██████████| 1945/1945 [04:16<00:00,  7.57it/s]


# Raw spectrogram combined

In [9]:
for t in ["accelerometer", "gyroscope"]:
    for desc, datafile, metafile in [("direct", "data.npz", "meta.csv"), ("train", "train.npz", "train.csv"), ("validation", "validation.npz", "validation.csv"), ("test", "test.npz", "test.csv")]:
        for nperseg, nooverlap, vmin, vmax in [(256, 200, 0.0, 1.0), (256, 200, 0.0, 0.05)]:
            descname = f"{desc}-{nperseg}-{nooverlap}-{vmin}-{vmax}"
            dataset_dir = os.path.join("data", "embbedings", "kuhar", "direct", "raw", t, "486c8aad")
            output_path = os.path.join("data", "embbedings", "kuhar", "spectrogram", "raw", f"{t}-combined", "486c8aad", descname) 
            data, metadata = read_data_meta(dataset_dir, datafile=datafile, metafile=metafile)

            remover_nans = RemoveNanAlongDimensions()
            splitter = SplitToLists()
            spec_extractor = ExtractCombinedSpectrogram(fs=100, nperseg=nperseg, nooverlap=nooverlap)
            plotter = PlotSpectrogram(vmin=vmin, vmax=vmax, figsize=(4,4), plot_axis=False, display=False)
            plot_and_save = PlotAndSave(plotter, output_path) 
            actions = [remover_nans, splitter, spec_extractor, plot_and_save]
            executor = ActionExecutor(
                description=f"Extract combined spectrogram from {t} to output: '{output_path}'",
                actions=actions,
                input_data=data,
                metadata=metadata
            )
            executor.run()

  0%|          | 2/1945 [00:00<02:11, 14.77it/s]

Executing 'Extract combined spectrogram from accelerometer to output: 'data/embbedings/kuhar/spectrogram/raw/accelerometer-combined/486c8aad/direct-256-200-0.0-1.0''


  0%|          | 2/1945 [00:00<02:22, 13.61it/s]

Executing 'Extract combined spectrogram from accelerometer to output: 'data/embbedings/kuhar/spectrogram/raw/accelerometer-combined/486c8aad/direct-256-200-0.0-0.05''


  0%|          | 1/216 [00:00<00:22,  9.48it/s]

...

  1%|          | 1/90 [00:00<00:09,  9.47it/s]

Executing 'Extract combined spectrogram from gyroscope to output: 'data/embbedings/kuhar/spectrogram/raw/gyroscope-combined/486c8aad/test-256-200-0.0-1.0''


  1%|          | 1/90 [00:00<00:09,  9.32it/s]

Executing 'Extract combined spectrogram from gyroscope to output: 'data/embbedings/kuhar/spectrogram/raw/gyroscope-combined/486c8aad/test-256-200-0.0-0.05''


100%|██████████| 90/90 [00:08<00:00, 11.24it/s]


# Time-Series Spectrogram

In [8]:
dataset_dir = os.path.join("data", "dataset_views", "kuhar", "time-series-300-samples-0-overlap", "accelerometer", "486c8aad")
output_path = os.path.join("outputs", "spectograms", "kuhar", "time-series-300-samples-0-overlap", "accelerometer-combined", "486c8aad", uid) 
data, metadata = read_data_meta(dataset_dir)
remover_nans = RemoveNanAlongDimensions()
splitter = SplitToLists()
spec_extractor = ExtractCombinedSpectrogram(fs=100, nperseg=20, nooverlap=10)
plotter = PlotSpectrogram(vmin=0.0, vmax=1.0, figsize=(4,4), plot_axis=False, display=False)
plot_and_save = PlotAndSave(plotter, output_path) 
actions = [remover_nans, splitter, spec_extractor, plot_and_save]
executor = ActionExecutor(
    description="Extract combined spectrogram from accelerometer",
    actions=actions,
    input_data=data,
    metadata=metadata
)
executor.run()

  0%|          | 3/19792 [00:00<14:57, 22.05it/s]

Executing 'Extract combined spectrogram from accelerometer'


100%|██████████| 19792/19792 [14:03<00:00, 23.46it/s]  


True

In [9]:
dataset_dir = os.path.join("data", "dataset_views", "kuhar", "time-series-300-samples-0-overlap", "gyroscope", "486c8aad")
output_path = os.path.join("outputs", "spectograms", "kuhar", "time-series-300-samples-0-overlap", "gyroscope-combined", "486c8aad", uid) 
data, metadata = read_data_meta(dataset_dir)

remover_nans = RemoveNanAlongDimensions()
splitter = SplitToLists()
spec_extractor = ExtractCombinedSpectrogram(fs=100, nperseg=20, nooverlap=10)
plotter = PlotSpectrogram(vmin=0.0, vmax=1.0, figsize=(4,4), plot_axis=False, display=False)
plot_and_save = PlotAndSave(plotter, output_path) 
actions = [remover_nans, splitter, spec_extractor, plot_and_save]
executor = ActionExecutor(
    description="Extract combined spectrogram from gyroscope",
    actions=actions,
    input_data=data,
    metadata=metadata
)
executor.run()

  0%|          | 3/19792 [00:00<12:11, 27.06it/s]

Executing 'Extract combined spectrogram from gyroscope'


100%|██████████| 19792/19792 [12:46<00:00, 25.81it/s]  


True

In [ ]:
data.shape + (2, )

In [ ]:
arr = np.zeros(data.shape + (2,))
for sample in range(data.shape[0]):
    for dimension in range(data.shape[1]):
        temp = np.fft.fft(data[sample, dimension])
        arr[sample, dimension, :, 0] = temp.real
        arr[sample, dimension, :, 1] = temp.imag

In [ ]:
x = arr[0,0].flatten()

In [ ]:
x.shape

In [ ]:
x